# Buidling an agent using Langchain

Basic building blocks:
* Tools: Tools in the context of Large Language Models refer to various functionalities and extensions that enhance the capabilities of these models beyond basic text generation. 
    * Langchain default tools
    * Custom tools
* agent: agent llm + prompt + invoke chain
* agent executor

**Ideas**
* what about a summarization tool? I'd like to see if we can 1) extract wiki content and then 2) summarize it
* Better wiki tool: QA retrieval chain: https://python.langchain.com/v0.1/docs/integrations/retrievers/wikipedia/
* this shows how to set up the wikipedia tool as custom tool: https://python.langchain.com/v0.1/docs/modules/tools/
* what is the difference between tools and openai functions


Use cases:
* create an agent that cna combine wikipedia data with relevant real-world information from API's. PRovide comprehensive answers tha encompass historical context and current data (eg weather, )
* create agent that can answer questions about specific topics by searching wiki, use NLP to analyze the articles content, summarize the key info and identify relevant sections, provide clear answers

In [ ]:
#Todo: dependency management

# !pip3 install langchain
# !pip3 install langchain_openai
# !pip3 install langchain_community
# pip install wikipedia
# pip install langchainhub

## A Langchain default tool: the [Wikipedia tool](https://python.langchain.com/v0.1/docs/integrations/tools/wikipedia/)


In [84]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1)#, doc_content_chars_max=1000)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [79]:
print(wiki_tool.name)
print(wiki_tool.description)
print(wiki_tool.args)
print(wiki_tool.return_direct)

wikipedia
A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
{'query': {'title': 'Query', 'type': 'string'}}
False


In [100]:
output = wiki_tool.run('pyladies')
output = wiki_tool.run('ruth bader ginsburg')

print(output)

Page: Ruth Bader Ginsburg
Summary: Joan Ruth Bader Ginsburg ( BAY-dər GHINZ-burg; née Bader; March 15, 1933 – September 18, 2020) was an American lawyer and jurist who served as an associate justice of the Supreme Court of the United States from 1993 until her death in 2020. She was nominated by President Bill Clinton to replace retiring justice Byron White, and at the time was viewed as a moderate consensus-builder. Ginsburg was the first Jewish woman and the second woman to serve on the Court, after Sandra Day O'Connor. During her tenure, Ginsburg authored the majority opinions in cases such as United States v. Virginia (1996), Olmstead v. L.C. (1999), Friends of the Earth, Inc. v. Laidlaw Environmental Services, Inc. (2000), and City of Sherrill v. Oneida Indian Nation of New York (2005). Later in her tenure, Ginsburg received attention for passionate dissents that reflected liberal views of the law. She was popularly dubbed "the Notorious R.B.G.", a moniker she later embraced.
Gins

In [99]:
type(output)

str

In [ ]:
# does not work

# from langchain_community.retrievers import WikipediaRetriever
# retriever = WikipediaRetriever()

# print(retriever.invoke('what is barbie?'))

# [Custom tools](https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/)

Key components:
* name (str), is required and must be unique within a set of tools provided to an agent
* description (str), is optional but recommended, as it is used by an agent to determine tool use
* args_schema (Pydantic BaseModel), is optional but recommended, can be used to provide more information (e.g., few-shot examples) or validation for expected parameters.
* what about a tool to save QA to a file

Tools can be defined in various ways, we will focus on the [StructuredTool dataclass](https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/#structuredtool-dataclass)

In [89]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
# from langchain.tools import BaseTool, StructuredTool, tool
from langchain.tools import StructuredTool

class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator_tool = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    # return_direct=True,
    # coroutine= ... <- you can specify an async method if desired as well
)

# Exercise: Write your own tool 

In [46]:
##

# Combine tools

In [90]:
# tools = [tool, calculator]
tools = [calculator_tool, wiki_tool]

# Define Agent in 3 lines

In [ ]:
# dependencies
from langchain_openai import ChatOpenAI # call openAI as agent llm
from langchain import hub # for the prompt, we are going to skip this
from langchain.agents import create_tool_calling_agent # set up the agent
from langchain.agents import AgentExecutor # execute agent

In [93]:

# agent llm
key = 'sk-proj-XX'
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key=key)

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages



[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [94]:
# agent definition
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Ryan Gosling`


Page: Ryan Gosling
Summary: Ryan Thomas Gosling ( GOSS-ling; born November 12, 1980) is a Canadian actor. Prominent in both independent films and major studio features, his films have grossed over $2 billion worldwide. Gosling has received various accolades, including a Golden Globe Award, and nominations for three Academy Awards and two British Academy Film Awards.
Gosling rose to prominence at age 13 on Disney Channel's The Mickey Mouse Club (1993–1995), and went on to appear in other family entertainment programs, including Are You Afraid of the Dark? (1995) and Goosebumps (1996). His breakthrough film role was that of a Jewish neo-Nazi in The Believer (2001), and he gained stardom in the 2004 romantic drama The Notebook. He starred in the critically acclaimed independent dramas Half Nelson (2006), for which he was nominated for the Academy Award for Best Actor; Lars and the Real Girl (2007), and Bl

{'input': 'who is Ryan Gosling?',
 'output': "Ryan Thomas Gosling is a Canadian actor known for his work in both independent films and major studio features. He has received various accolades, including a Golden Globe Award, and nominations for three Academy Awards and two British Academy Film Awards. Gosling rose to prominence at age 13 on Disney Channel's The Mickey Mouse Club and has since starred in a variety of films, including The Notebook, Half Nelson, La La Land, and Blade Runner 2049. He is also a musician and a co-owner of a Moroccan restaurant in Beverly Hills. Gosling is involved in various charitable causes and is in a relationship with actress Eva Mendes, with whom he has two daughters."}

In [108]:
# agent_executor.invoke({"input": "who is Ryan Gosling?"})
# agent_executor.invoke({"input": "what is three times 5"})
question = "How old is Ryan gosling?"
question = 'how many children did ruth bader ginsburg have?'
question = 'write me a short summary about ruth bader ginsburg'
question = 'what was ruth bader ginsburg passionate about?'
question = 'who were ruth baeder ginsburgs colleagues at the supreme court?'
question = 'what are great holiday destinations?'
question = 'which city is bigger: Paris or Munich?'
question = 'what is mollie?'
agent_executor.invoke({"input": question})




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Mollie`




/Users/maria/Library/Python/3.9/lib/python/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/maria/Library/Python/3.9/lib/python/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


No good Wikipedia Search Result was foundI couldn't find any specific information about "Mollie" on Wikipedia. Would you like me to search for more general information or try to answer a different question related to Mollie?

> Finished chain.


{'input': 'what is mollie?',
 'output': 'I couldn\'t find any specific information about "Mollie" on Wikipedia. Would you like me to search for more general information or try to answer a different question related to Mollie?'}

# the magic behind the agent [link](https://api.python.langchain.com/en/latest/_modules/langchain/agents/tool_calling_agent/base.html#create_tool_calling_agent)

# Sources
* https://python.langchain.com/docs/modules/agents/quick_start/